# Septin Analysis: comparing the mean shape of each group

# 1. Overview of analysis

First, we will

# 2. Imports

## 2.1 Notebook setup

In [9]:
import os
import subprocess
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs

geomstats_gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(geomstats_gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings
warnings.filterwarnings("ignore")

Working directory:  /Users/adelemyers/code/dyn


## 2.2 Standard Imports

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [11]:
from geomstats.learning.pca import TangentPCA

from scipy import stats

from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn import manifold
from joblib import Parallel, delayed
from numba import jit, njit, prange

## 2.3 Project-specific imports

In [12]:
from geomstats.geometry.discrete_curves import R2, DiscreteCurves, ClosedDiscreteCurves

from geomstats.learning.frechet_mean import FrechetMean
from geomstats.learning.kmeans import RiemannianKMeans

import dyn.dyn.datasets.experimental as experimental
import dyn.dyn.features.basic as basic
import dyn.viz as viz

viz.init_matplotlib()

# 3. Load the datasets of interest

## 3.1 testing the method i will add to experiemental.py

In [44]:
import importlib
importlib.reload(experimental)

import dyn.dyn.datasets.experimental as experimental


def _tif_video_to_lists(tif_path):
    """Convert a cell video into two trajectories of contours and images.

    Parameters
    ----------
    tif_path : absolute path of video in .tif format.

    Returns
    -------
    contours_list : list of arrays
        List of 2D coordinates of points defining the contours of each cell
        within the video.
    imgs_list : list of array
        List of images in the input video.
    """
    img_stack = skio.imread(tif_path, plugin="tifffile")
    contours_list = []
    imgs_list = []
    for img in img_stack:
        imgs_list.append(img)
        thresh = threshold_otsu(img)
        binary = img > thresh
        contours = measure.find_contours(binary, 0.8)
        lengths = [len(c) for c in contours]
        max_length = max(lengths)
        index_max_length = lengths.index(max_length)
        contours_list.append(contours[index_max_length])

    return contours_list, imgs_list


n_sampling_points=10
n_cells=-1

list_tifs = experimental.load_septin_cells(n_cells,n_sampling_points)

n_traj = len(list_tifs)
print(n_traj)
one_img_stack = skio.imread(list_tifs, plugin="tifffile")
print(one_img_stack.shape)
n_time_points, height, width = one_img_stack.shape

centers_traj = gs.zeros((n_traj, n_time_points, 2))
shapes_traj = gs.zeros((n_traj, n_time_points, n_sampling_points, 2))
imgs_traj = gs.zeros((n_traj, n_time_points, height, width))
labels = []
#for i_traj, video_path in enumerate(list_tifs):
for video_path in enumerate(list_tifs):
    video_name = os.path.basename(video_path)
    print(f"\n Processing trajectory {i_traj+1}/{n_traj}.")

    print(f"Converting {video_name} into list of cell contours...")
    contours_list, imgs_list = _tif_video_to_lists(video_path)

    labels.append(int(video_name.split("_")[0]))
    for i_contour, (contour, img) in enumerate(zip(contours_list, imgs_list)):
        interpolated = _interpolate(contour, n_sampling_points)
        cleaned = _remove_consecutive_duplicates(interpolated)
        center = gs.mean(cleaned, axis=-2)
        centered = cleaned - center[..., None, :]
        centers_traj[i_traj, i_contour] = center
        shapes_traj[i_traj, i_contour] = centered
        if img.shape != (height, width):
            print(
                "Found image of a different size: "
                f"{img.shape} instead of {height, width}. "
                "Skipped image (not cell contours)."
            )
            continue
        imgs_traj[i_traj, i_contour] = gs.array(img.astype(float).T)
labels = gs.array(labels)


36
(36, 512, 512)


TypeError: expected str, bytes or os.PathLike object, not tuple

I think that the issue here is that all of this code is meant for something where the shape of list_tifs is bigger than the example case.

yes, so what is happening is that it think that each file in the example is its own "movie"/set of trajectories, and in our case, we are considering just a bunch of single images. so the example list_tifs has an extra dimension. so we shouldnt even be considering the variable n_tifs

In [45]:
import importlib
importlib.reload(experimental)
import dyn.dyn.datasets.experimental as experimental

centers_traj, shapes_traj, imgs_traj, labels = experimental.load_trajectory_of_border_cells(
    n_sampling_points=50)

16
(25, 512, 512)

 Processing trajectory 1/16.
Converting 104438_MAX_9-7.tif into list of cell contours...

 Processing trajectory 2/16.
Converting 33623_MAX_9-2-1-28.tif into list of cell contours...
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (190, 347) instead of (512, 512). Skipped image (no


 Processing trajectory 7/16.
Converting 59080_MAX_9-7.tif into list of cell contours...
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (287, 443) instead of (512, 512). Skipped image (not cell cont

Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (not cell contours).
Found image of a different size: (278, 418) instead of (512, 512). Skipped image (